# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

## 5. Data Modeling
**Bài toán**: Dự đoán một công ty đã nằm trong bảng xếp hạng FT1000 có tiếp tục nằm trong bảng xếp hạng FT1000 năm sau hay không?

**Ý nghĩa**: Cùng ý nghĩa với các câu hỏi ở phần Data Analyzing, khi biết được một công ty đã được xếp hạng trong FT1000 năm nay có tiếp tục nằm trong bảng xếp hạng FT1000 vào năm sau hay không giúp các nhà đầu tư đưa ra lựa chọn đầu tư chính xác hơn, bên cạnh các chỉ số về doanh thu hay mức tăng trưởng đã được phân tích trong bảng xếp hạng năm nay

## 5.1. Thu thập thêm dữ liệu
Dữ liệu bảng xếp hạng năm nay là không đủ để  giải quyết bài toán, ta cần thu thập dữ liệu của cả những bảng xếp hạng trước. Bảng xếp hạng FT tính đến namư 2022 được tổ chức lần thứ 6, do đó ta cần thu thập thêm dữ liệu từ 5 bảng xếp hạng ở 5 năm trước. Phương pháp thu thập cũng sẽ giống việc thu thập ở phần Data Collecting

In [ ]:
url = 'https://ig.ft.com/ft-1000/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2017 = pd.DataFrame(js[0])
df2017.rename(columns={'Unnamed: 0':'Rank'}, inplace = True)
df2017 = df2017.sort_values(by = 'Rank').reset_index(drop = True)
df2017.head()

,Rank,Company,Country,Sector,RevenueGrowth 2012-5,CAGR2012-5,Revenue2015 (’000 €),Employees2015,Founded
0,1,HelloFresh,Germany,Food & Beverage,"13,159%",409.9%,304952,981.0,2011
1,2,Codewise,Poland,Advertising,"13,052%",408.5%,31613,103.0,2011
2,3,Green IT Das Systemhaus,Germany,Technology,"11,113%",382.2%,24305,80.0,2012
3,4,Brainlabs,United Kingdom,Advertising,"8,218%",336.5%,13608,48.0,2012
4,5,Catapult Enterprises (Propercorn),United Kingdom,Food & Beverage,"7,510%",323.8%,12052,32.0,2011


In [ ]:
url = 'https://www.ft.com/content/cf0c5fce-3112-11e8-b5bf-23cb17fd1498'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2018 = pd.DataFrame(js[0])
df2018.head()

,Rank,Company,Location,Sector,Revenue growth 2013-6 (%),CAGR 2013-6 (%),Revenue 2016 (€'000),Employee growth 2013-6 (%),Employees 2016,Founded
0,1,Deliveroo*,United Kingdom,Food & Beverage,107117,923.5,157406,1046.0,1049.0,2013
1,2,Thermondo,Germany,Energy,10878,378.8,20222,280.0,300.0,2012
2,3,Traventia Viajes*,Spain,Travel & Leisure,9377,355.9,10295,19.0,20.0,2013
3,4,Alainsa,Spain,Construction,9114,351.7,10286,19.0,20.0,1980
4,5,iTravex,Spain,Travel & Leisure,8398,339.6,22674,28.0,37.0,2007


In [ ]:
url = 'https://www.ft.com/content/238174d2-3139-11e9-8744-e7016697f225'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2019 = pd.DataFrame(js[0])
df2019.head()

,Rank,Company,In 2018 list?,In 2017 list?,Country,Sector,Absolute revenue growth,Revenue CAGR 2014-17,Revenue 2017 (€m),Employee growth 2014-17,Employees 2017,Founded
0,1,Blue Motor Finance,yes,no,United Kingdom,Financial Services,"51,364%",701.4%,61.4,143.0,155.0,1992
1,2,Deliveroo,yes,no,United Kingdom,Food & Beverage,"15,749%",441.2%,316.1,1636.0,1664.0,2013
2,3,Taxify,no,no,Estonia,Technology,"12,231%",397.7%,17.8,336.0,350.0,2013
3,4,Solectric,no,no,Germany,Technology,"7,772%",328.6%,40.9,27.0,30.0,2012
4,5,Psioxus Therapeutics,no,no,United Kingdom,Pharmaceuticals,"7,126%",316.5%,64.4,41.0,69.0,2006


In [ ]:
url = 'https://www.ft.com/content/691390ca-53d9-11ea-90ad-25e377c0ee1f'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2020 = pd.DataFrame(js[0])
df2020.head()

,Rank,Name,in 2019 ranking,in 2018 ranking,Country,FT Category,Absolute Growth Rate [in %],Compound Annual Growth Rate (CAGR) [in %],Revenue 2018 [in €m],Revenue 2015 [in €m],Number of employees 2018,Founding Year
0,1,OakNorth Bank,no,no,United Kingdom,Fintech,"37,462.5%",621.5%,67.91,0.22,200.0,2015
1,2,Wolt Enterprises,no,no,Finland,Technology,"15,641.9%",439.9%,30.07,0.19,293.0,2014
2,3,Bolt Technology,yes,no,Estonia,Technology,"12,959.7%",407.4%,79.68,0.61,700.0,2013
3,4,Elements Global Services,no,no,Spain,Support Services,"10,233.3%",369.3%,26.25,0.27,38.0,2015
4,5,Les Eco-Isolateurs,yes,no,France,Construction,"10,077.5%",366.9%,32.98,0.32,67.0,2013


In [ ]:
url = 'https://www.ft.com/content/8b37a92b-15e6-4b9c-8427-315a8b5f4332'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2021 = pd.DataFrame(js[0])
df2021.iloc[:, :13].head()

,Rank,Name,in 2020 ranking,in 2019 ranking,Country,FT Category,Absolute Growth Rate %,Compound Annual Growth Rate (CAGR) %,Revenue 2019 €,Revenue 2016 €,Number of employees 2019,Number of employees 2016,Founding Year
0,1,Bulb Energy,No,No,UK,Energy,199626.7,1159.3,1737586657,985867,575,55,2015
1,2,Sun Finance,No,No,Latvia,Fintech,61837.8,752.4,101475796,163835,750,3,2012
2,3,Everflow Group,No,No,UK,Energy,17940.4,465.0,55902990,363266,46,4,2015
3,4,Glencar Construction,No,No,UK,Construction,15735.4,441.0,110298923,724551,85,2,2015
4,5,Spyrosoft,No,No,Poland,Technology,14783.8,430.0,16635991,110092,376,12,2016


In [ ]:
df2022 = pd.read_csv('data/FT1000_v1.csv')
df2022.head()

,Rank,Name,Ranked2021,Ranked2020,Country,Sector,CAGR,Revenue2020,Revenue2017,Employees2020,Employees2017,FoundingYear,Country1,Sector1
0,1,Swappie,No,No,Finland,Technology,477.43,97611814,507000,218,1,2016,Finland,Technology
1,2,Kilo Health,No,No,Lithuania,Health,450.05,57318766,344428,177,10,2013,Lithuania,Health
2,3,OCI,No,No,UK,Financial Services,409.59,568322073,4325512,32,4,2012,UK,Financial Services
3,4,OnlyFans,No,No,UK,Technology,393.63,316732986,2652185,800,4,2016,UK,Technology
4,5,Enpal,No,No,Germany,Energy,386.88,56109613,486165,365,9,2017,Germany,Energy
